In [1]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

### Get data from ChEMBL

Filter them directly when requesting them from the server.

In [2]:
dr = new_client.drug.filter(first_approval__isnull=False, first_approval__gte=2012.0).order_by('first_approval')
drdf = pd.DataFrame(dr)
drdf.head()

,applicants,atc_classification,availability_type,biotherapeutic,black_box,black_box_warning,chirality,development_phase,drug_type,first_approval,...,usan_stem,usan_stem_definition,usan_stem_substem,usan_year,withdrawn_class,withdrawn_country,withdrawn_flag,withdrawn_reason,withdrawn_year,atc_code_description
0,[Genentech Inc],"[{'code': 'L01XJ01', 'description': 'ANTINEOPL...",1,None,True,1,2,4,1,2012,...,-degib,hedgehog signaling inhibitors,-degib(-degib),2009.0,None,None,0,None,None,NaN
1,"[Allergan Sales Llc, Mylan Pharmaceuticals Inc]","[{'code': 'A06AX04', 'description': 'ALIMENTAR...",1,"{'biocomponents': [], 'description': 'LINACLOT...",True,1,1,4,5,2012,...,-tide,peptides: gastrointestinal functions; normalis...,-tide(-tide (-clotide)),2005.0,None,None,0,None,None,NaN
2,None,"[{'code': 'J06BB18', 'description': 'ANTIINFEC...",1,"{'biocomponents': [], 'description': 'Raxibacu...",False,0,1,4,6,2012,...,-mab,"monoclonal antibodies: fully human, cardiovasc...",-mab(-mab (-cumab)),2004.0,None,None,0,None,None,NaN
3,[Gilead Sciences Inc],"[{'code': 'J05AJ02', 'description': 'ANTIINFEC...",1,None,False,0,1,4,1,2012,...,-vir,antivirals: integrase inhibitors,-vir(-vir (-tegravir)),2007.0,None,None,0,None,None,NaN
4,[Avid Radiopharmaceuticals Inc],"[{'code': 'V09AX05', 'description': 'VARIOUS: ...",1,None,False,0,2,4,1,2012,...,None,None,None,2009.0,None,None,0,None,None,NaN


In [3]:
drdf.columns

Index(['applicants', 'atc_classification', 'availability_type',
       'biotherapeutic', 'black_box', 'black_box_warning', 'chirality',
       'development_phase', 'drug_type', 'first_approval', 'first_in_class',
       'helm_notation', 'indication_class', 'molecule_chembl_id',
       'molecule_properties', 'molecule_structures', 'molecule_synonyms',
       'ob_patent', 'oral', 'parenteral', 'prodrug', 'research_codes',
       'rule_of_five', 'sc_patent', 'synonyms', 'topical', 'usan_stem',
       'usan_stem_definition', 'usan_stem_substem', 'usan_year',
       'withdrawn_class', 'withdrawn_country', 'withdrawn_flag',
       'withdrawn_reason', 'withdrawn_year', 'atc_code_description'],
      dtype='object')

Get activities and link drugs to targets.

In [4]:
acts = new_client.activity.filter(molecule_chembl_id__in=list(drdf.molecule_chembl_id)).only(['target_chembl_id'])
targets_id = [a['target_chembl_id'] for a in acts]

In [5]:
print(len(targets_id))
targets_id_unique = list(set(targets_id))
print(len(targets_id_unique))

60437
2605


In [ ]:
targs = new_client.target.filter(targets_chembl_id__in=targets_id_unique)
tgdf = pd.DataFrame(targs)
tgdf.head()
tgdf.shape

<span style="color:red">I do not get why this dataframe has over 14'000 entries when I filtered them according to a target ID list of length 2600.</span>

In [ ]:
accessions = []
missing = 0
for t in targs:
    if len(t['target_components']) > 0:
        accessions.append(t['target_components'][0]['accession'])
    else:
        missing += 1

In [ ]:
print(len(accessions))
accessions_unique = list(set(accessions))
print(len(accessions_unique))

<span style="color:red">Why are there duplicates in accession numbers?</span>

### Get data from UniProt

In [ ]:
import requests, sys

In [ ]:
requestURLs = ["https://www.ebi.ac.uk/proteins/api/proteins/{}".format(a) for a in accessions[:10]]
# TODO: resquest for all accession numbers instead of just the first ones once it works

keywords = []

for url in requestURLs:
    r = requests.get(url, headers={"Accept": "application/xml"})
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    responseBody = r.text
    keywords.append(responseBody)

print(keywords[0])

<span style="color:red">Not sure what is meant with UniProt keywords.</span>